# Prescription Parser using CRF
Builds a Doctor Prescription Parser with the help of CRF model

Builds a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
!pip install sklearn-crfsuite

#### Import the necessary libraries

In [2]:
# For CRF model
import sklearn_crfsuite
import joblib

from sklearn.model_selection import train_test_split
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/moossa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(inp, out):
    sample_data = []
    
    # Iterate over each sentence and its corresponding labels
    for tokens, labels in zip(inp, out):
        # Pair each token with its label and create a list of tuples
        sentence_data = list(zip(tokens, labels))
        # Append the result for this sentence to the final list
        sample_data.append(sentence_data)    
    return sample_data

In [6]:
whole_data=tuples_maker(input_sigs, output_labels)
whole_data

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [7]:
def triples_maker(whole_data):
    sample_data = []
    
    for sentence in whole_data:
        # Extract tokens and labels separately
        tokens, labels = zip(*sentence)
        
        # Get POS tags for the tokens
        pos_tags = nltk.pos_tag(tokens)
        
        # Form triples of (token, POS, label) and append to sample_data
        sentence_data = [(token, pos, label) for (token, pos), label in zip(pos_tags, labels)]
        sample_data.append(sentence_data)    
    return sample_data

In [8]:
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag

In [9]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1] if len(doc[i]) > 1 else "UNK"  # Default to "UNK" if postag is missing

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1] if len(doc[i-1]) > 1 else "UNK"
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1] if len(doc[i+1]) > 1 else "UNK"
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [10]:
def sent2features(sent):
    return [token_to_features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for label in sent]

train_sigs, test_sigs, train_labels, test_labels = train_test_split(input_sigs, output_labels, test_size=0.2, random_state=42)

### Training the CRF model with the features extracted using the feature extractor method

In [11]:


# Submit training data to the trainer
X_train = [sent2features(s) for s in train_sigs]
y_train = [sent2labels(s) for s in train_labels]

X_test = [sent2features(s) for s in test_sigs]
y_test = [sent2labels(s) for s in test_labels]

# Set the parameters of the model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished

joblib.dump(crf, 'crf_model.pkl')  # Save the trained model to a file

['crf_model.pkl']

### Predicting the test data with the built model

In [12]:
# Predicting on the test data
y_pred = crf.predict(X_test)

# Predictions of 5 sentences
for i, sentence_pred in enumerate(y_pred[:5]):  
    print(f"Sentence {i+1} Predictions: {sentence_pred}")


Sentence 1 Predictions: ['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']
Sentence 2 Predictions: ['EVERY', 'Period', 'PeriodUnit']
Sentence 3 Predictions: ['WHEN']
Sentence 4 Predictions: ['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit']
Sentence 5 Predictions: ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Flatten the sequences of labels
y_test_flat = [label for sublist in y_test for label in sublist]
y_pred_flat = [label for sublist in y_pred for label in sublist]

# MultiLabelBinarizer to convert the labels into a binary format
mlb = MultiLabelBinarizer()
y_test_bin = mlb.fit_transform([[label] for label in y_test_flat])
y_pred_bin = mlb.transform([[label] for label in y_pred_flat])

report = classification_report(y_test_bin, y_pred_bin, target_names=mlb.classes_, zero_division=0)

print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

         AND       0.00      0.00      0.00         1
          AT       1.00      1.00      1.00         1
      BEFORE       0.67      1.00      0.80         2
         BID       0.00      0.00      0.00         2
    Duration       1.00      0.75      0.86         4
 DurationMax       0.00      0.00      0.00         1
DurationUnit       1.00      0.50      0.67         4
       EVERY       1.00      1.00      1.00         3
         FOR       1.00      0.75      0.86         4
        Form       0.83      1.00      0.91         5
   Frequency       0.33      1.00      0.50         1
      Method       1.00      1.00      1.00         3
          PO       0.00      0.00      0.00         2
      Period       0.80      1.00      0.89         4
   PeriodMax       0.50      1.00      0.67         1
  PeriodUnit       0.57      1.00      0.73         4
         Qty       0.83      1.00      0.91         5
    

/opt/anaconda3/envs/spacy_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['DAILY'] will be ignored
  warnings.warn(


### Putting all the prediction logic inside a predict method

In [14]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    # Tokenize the sentence
    tokens = sig.split()
    
    # Extract features for each token
    features = [sent2features(tokens)]
    
    # Use the CRF model to predict the labels
    predictions = crf.predict(features)

    # Extract the first element and convert it to a list
    label_list = [list(predictions[0])]
    
    
    # Print the sentence
    print(sig)
    print(label_list)
    
    return sig, label_list

### Sample predictions

In [15]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [16]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [17]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [18]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [19]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [20]:
predictions = predict("x 30 days")

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [21]:
predictions = predict("x 20 months")

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [22]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [23]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [24]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'PO', 'TID']]


In [25]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'Frequency', 'BY', 'PO']]


In [26]:
predictions = predict("inject 3 units orally")

inject 3 units orally
[['Method', 'Qty', 'Form', 'Frequency']]


In [27]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
[['Qty', 'Form', 'Qty', 'Form', 'TID']]


In [28]:
predictions = predict("by mouth take three caps")

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [29]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [30]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'BEFORE', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [31]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BEFORE', 'WHEN', 'AT', 'WHEN']]


In [32]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [33]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Period', 'PeriodUnit']]


In [34]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Qty', 'Method', 'Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [35]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['Method', 'Qty', 'Qty', 'Form']]


In [36]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [37]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit']]


In [38]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'Period', 'FOR', 'Duration', 'DurationUnit']]


In [39]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [40]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [41]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [42]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [43]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [44]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [45]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']]


In [46]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [47]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [48]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [49]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [50]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [51]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [52]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [53]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [54]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'Period', 'EVERY', 'Period', 'PeriodUnit']]


In [55]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['Qty', 'Form', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [56]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [57]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [58]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
